In [20]:
!pip install torchmetrics


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: python -m pip install --upgrade pip


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error as  mape
from matplotlib import pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, Lasso, LassoCV

from torchmetrics import MeanSquaredLogError


In [22]:
from preprocess import EstateData

dataset = EstateData("final.csv")

In [23]:
dataset.split_data(stratify=True)

In [24]:
dataset.preprocess(tukey=True)

Start Process!
Fill missing values: Done
Remove outlier: Done
Scale features: Done


In [25]:
train = dataset.train
test = dataset.test

In [26]:
print("train size {}".format(train.shape))
print("test size {}".format(test.shape))

train size (31531, 13)
test size (9278, 13)


In [27]:
train, dev = train_test_split(train, test_size=0.1)

In [28]:
train = pd.get_dummies(train)
dev = pd.get_dummies(dev)
test = pd.get_dummies(test)

In [29]:
features = train.columns.tolist()
features.remove("Price")
features.remove("PricePerM2")

In [30]:
linear_re = LinearRegression()


In [31]:
linear_re.fit(train[features], train["PricePerM2"])



LinearRegression()

In [32]:
pred = linear_re.predict(dev[features])
print(mape(dev["PricePerM2"], pred))

1.8703360128776885


In [33]:
pred = linear_re.predict(test[features])
print(mape(test["PricePerM2"], pred))

2.2990097562997343


In [46]:
lasso = Lasso(alpha=0.1)
lasso.fit(train[features], train["PricePerM2"])

Lasso(alpha=0.1)

In [47]:
pred = lasso.predict(dev[features])
print(mape(dev["PricePerM2"], pred))

1.8656527769557667


In [48]:
pred = lasso.predict(test[features])
print(mape(test["PricePerM2"], pred))

2.2921317276709354


In [57]:
lassoCV = LassoCV(cv=5, random_state=0)
lassoCV.fit(train[features], train["PricePerM2"])

LassoCV(cv=5, random_state=0)

In [58]:
pred = lassoCV.predict(dev[features])
print(mape(dev["PricePerM2"], pred))

1.862492642555701


In [59]:
pred = lassoCV.predict(test[features])
print(mape(test["PricePerM2"], pred))

2.289284027816132
